---
# --- Day 22: Crab Combat ---
---

In [122]:
import numpy as np

### Input

In [171]:
with open("data/22_input.txt") as f:
    data = [l.strip() for l in f.readlines()]

In [172]:
def interpret_data(data):
    empty_idx = [i for i, l in enumerate(data) if l == ""][0]
    deck1 = [int(n) for n in data[1:empty_idx]]
    deck2 = [int(n) for n in data[empty_idx+2:]]
    return deck1, deck2

In [173]:
deck1, deck2 = interpret_data(data)

### Part 1: find winner and compute score

In [174]:
def play_the_game(deck1, deck2):
    d1 = deck1.copy()
    d2 = deck2.copy()
    history = {}
    loop_encountered = False
    while (len(d1) > 0) and (len(d2) > 0) and (not loop_encountered):
        history[len(d1)] = history.get(len(d1), []) + [(d1, d2)]
        top1 = d1[0]
        queue1 = d1[1:] if len(d1) > 1 else []
        top2 = d2[0]
        queue2 = d2[1:] if len(d2) > 1 else []
        if top1 > top2:
            d1 = queue1 + [top1, top2]
            d2 = queue2
        elif top1 < top2 :
            d1 = queue1
            d2 = queue2 + [top2, top1]
        else:
            d1 = queue1 + [top1]
            d2 = queue2 + [top2]
        if np.any([np.allclose(d1, h[0]) and np.allclose(d2, h[1]) for h in history.get(len(d1), [])]):
            loop_encountered = True         
    winner = 1 if loop_encountered else int(len(d1) == 0) + 1        
    return d1, d2, winner

In [175]:
def compute_score(d1, d2):
    winning_deck = d2 if len(d1) == 0 else d1
    n = len(winning_deck)
    winning_deck = np.array(winning_deck).reshape((n, 1))
    weights = np.flip(np.arange(n)+1).reshape((1, n))
    score = weights.dot(winning_deck)[0][0]
    return score

In [176]:
d1, d2, winner = play_the_game(deck1, deck2)
compute_score(d1, d2)

32272

### Part 2: recursive combat

In [178]:
def recursive_combat(deck1, deck2):
    d1 = deck1.copy()
    d2 = deck2.copy()
    history = {}
    loop_encountered = False
    i = 0
    while (len(d1) > 0) and (len(d2) > 0) and (not loop_encountered):
        history[len(d1)] = history.get(len(d1), []) + [(d1, d2)]
        top1 = d1[0]
        queue1 = d1[1:] if len(d1) > 1 else []
        top2 = d2[0]
        queue2 = d2[1:] if len(d2) > 1 else []
        # round turning into a sub-game
        if (len(queue1) >= top1) and (len(queue2) >= top2):
            sd1, sd2, winner = play_the_game(queue1[:top1], queue2[:top2])
        # normal round
        else:
            winner = 1 if top1 > top2 else 2 if top2 > top1 else 0
        if winner == 1:
            d1 = queue1 + [top1, top2]
            d2 = queue2
        elif winner == 2:
            d1 = queue1
            d2 = queue2 + [top2, top1]
        else:
            d1 = queue1 + [top1]
            d2 = queue2 + [top2]
        if np.any([np.allclose(d1, h[0]) and np.allclose(d2, h[1]) for h in history.get(len(d1), [])]):
            loop_encountered = True   
    final_winner = int(len(d1) == 0) + 1
    if loop_encountered:
        print(f"Game stopped after {i} rounds to prevent loops. Player 1 won.")
    else:
        print(f"Game ended normally. Player {final_winner} won.")
    return d1, d2, history

In [179]:
d1, d2, history = recursive_combat(deck1, deck2)

Game ended normally. Player 1 won.


In [180]:
compute_score(d1, d2)

33206